## Create Visual Genome dataset

In [1]:
import sys, os
sys.path.append("../")
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [2]:
import os, shutil, gzip, json
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.shared import handle_uploaded_file
from dvaapp import models
from dvaapp.models import TEvent
from dvaapp.tasks import extract_frames, export_video_by_id
from collections import defaultdict

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


### Download data from AWS Requester Pays bucket
####(Note that this will cost 1.5$ outside US-East-1 AWS region)

In [ ]:
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key visual_genome_objects.txt.gz  /root/DVA/visual_genome_objects.txt.gz
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key visual_genome.zip  /root/DVA/visual_genome.zip

In [3]:
data = defaultdict(list)
with gzip.open('/root/DVA/visual_genome_objects.txt.gz') as metadata:
    for line in metadata:
        entries = line.strip().split('\t')
        data[entries[1]].append({
            'x':int(entries[2]),
            'y':int(entries[3]),
            'w':int(entries[4]),
            'h':int(entries[5]),
            'object_id':entries[0],
            'object_name':entries[6],
            'metadata_text':' '.join(entries[6:]),})

In [4]:
name = "visual_genome"
fname = "visual_genome.zip"
f = SimpleUploadedFile(fname, "", content_type="application/zip")
v = handle_uploaded_file(f, name)

In [5]:
import shutil
shutil.copy("/root/DVA/visual_genome.zip",'/root/DVA/dva/media/{}/video/{}.zip'.format(v.pk,v.pk))

In [6]:
extract_frames(TEvent.objects.create(video=v).pk)

0

In [7]:
video = v
models.Region.objects.all().filter(video=video).delete()
buffer = []
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('/')[-1].split('.')[0]))
    for o in data[frame_id]:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video = v
        annotation.frame = frame
        annotation.x = o['x']
        annotation.y = o['y']
        annotation.h = o['h']
        annotation.w = o['w']
        annotation.object_name = o['object_name']
        annotation.metadata = o
        annotation.metadata_text = o['metadata_text']
        buffer.append(annotation)
        if len(buffer) == 1000:
            try:
                models.Region.objects.bulk_create(buffer)
                print "saving"
            except:
                print "encountered an error doing one by one"
                for k in buffer:
                    try:
                        k.save()
                    except:
                        print "skipping"
                        print k.object_name                
            buffer = []
try:
    models.Region.objects.bulk_create(buffer)
    print "saving"
except:
    print "encountered an error doing one by one"
    for k in buffer:
        try:
            k.save()
        except:
            print "skipping"
            print k.object_name

saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving

saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving

saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving

saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving
saving

In [ ]:
# export_video_by_id(TEvent.objects.create(video=v).pk)